<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_RESTful_Exemplos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acesso a uma RESTful API

Aqui está um exemplo básico de como você pode escrever um código em Python para acessar uma REST API de uma loja de departamentos.

Certifique-se de ter a biblioteca requests instalada. Você pode instalá-la usando o seguinte comando:

In [ ]:
!pip install requests

Neste exemplo, vou supor que a API fornece informações sobre produtos disponíveis na loja.

In [ ]:
import requests

# URL base da API da loja de departamentos
base_url = "https://api.lojadepartamentos.com"

# Função para obter a lista de produtos da API
def get_product_list():
    url = f"{base_url}/products"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve product list. Status code: {response.status_code}")
        return None

# Função para obter detalhes de um produto específico
def get_product_details(product_id):
    url = f"{base_url}/products/{product_id}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve product details. Status code: {response.status_code}")
        return None

# Obtendo e exibindo a lista de produtos
product_list = get_product_list()
if product_list:
    print("List of Products:")
    for product in product_list:
        print(f"ID: {product['id']}, Name: {product['name']}, Price: {product['price']}")

# Obtendo e exibindo os detalhes de um produto específico
product_id = 1  # Substitua pelo ID do produto desejado
product_details = get_product_details(product_id)
if product_details:
    print("\nProduct Details:")
    print(f"ID: {product_details['id']}")
    print(f"Name: {product_details['name']}")
    print(f"Price: {product_details['price']}")
    print(f"Description: {product_details['description']}")


# Criar uma RESTFul API para Acesso

Vou fornecer um exemplo básico de como criar uma API usando a biblioteca FastAPI em Python para acessar um banco de dados de uma universidade. Neste exemplo, vou usar o SQLite como banco de dados para simplificar, mas você pode adaptá-lo para o banco de dados de sua escolha.

Certifique-se de ter a biblioteca FastAPI e o SQLAlchemy instalados:

In [ ]:
!pip install fastapi[all] sqlalchemy

Aqui está um exemplo de como você pode criar uma API para acessar informações de alunos em um banco de dados:

In [ ]:
from fastapi import FastAPI, HTTPException
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

app = FastAPI()

# Definindo o modelo do aluno usando SQLAlchemy
Base = declarative_base()

class Student(Base):
    __tablename__ = "students"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    age = Column(Integer)

# Configurando o banco de dados SQLite
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Rota para obter informações de um aluno por ID
@app.get("/students/{student_id}")
async def get_student(student_id: int):
    db = SessionLocal()
    student = db.query(Student).filter(Student.id == student_id).first()
    db.close()

    if student is None:
        raise HTTPException(status_code=404, detail="Student not found")

    return student

# Rota para criar um novo aluno
@app.post("/students/")
async def create_student(student: Student):
    db = SessionLocal()
    db.add(student)
    db.commit()
    db.refresh(student)
    db.close()

    return student

# Criar uma RESTful API com Acesso Seguro via Token

Vou mostrar um exemplo básico de como criar uma API em Python usando o FastAPI e implementar a segurança de acesso via token utilizando tokens JWT (JSON Web Tokens).

Certifique-se de ter a biblioteca FastAPI, a biblioteca PyJWT e a biblioteca passlib instaladas:

In [ ]:
!pip install fastapi[all] pyjwt passlib

Aqui está um exemplo de como você pode criar uma API segura com autenticação por token JWT:

In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer
from passlib.context import CryptContext
from jose import JWTError, jwt
from datetime import datetime, timedelta

app = FastAPI()

# Configuração básica
SECRET_KEY = "your-secret-key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Simulando um banco de dados de usuários
fake_users_db = {
    "testuser": {
        "username": "testuser",
        "password": "$2b$12$SNXlOkoUdAeYSIw8NPKw9un2fr8WZRUq51gqtNodTruhUSskF3uDS",  # Hashed "testpassword"
    }
}

class TokenData:
    def __init__(self, username: str):
        self.username = username

class UserInDB:
    def __init__(self, **user_data):
        self.__dict__.update(user_data)

# Funções auxiliares de autenticação
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)

def get_password_hash(password):
    return pwd_context.hash(password)

# Funções relacionadas a autenticação via token JWT
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

def create_access_token(data: dict, expires_delta: timedelta):
    to_encode = data.copy()
    expire = datetime.utcnow() + expires_delta
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

def decode_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None:
            raise JWTError("Subclaim missing")
        token_data = TokenData(username=username)
        return token_data
    except JWTError:
        raise HTTPException(status_code=401, detail="Invalid token")

# Rotas
@app.post("/token")
async def login_for_access_token(username: str, password: str):
    user = fake_users_db.get(username)
    if user is None or not verify_password(password, user["password"]):
        raise HTTPException(status_code=401, detail="Invalid credentials")
    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(data={"sub": username}, expires_delta=access_token_expires)
    return {"access_token": access_token, "token_type": "bearer"}

@app.get("/secure-data")
async def get_secure_data(token: str = Depends(oauth2_scheme)):
    token_data = decode_token(token)
    user = fake_users_db.get(token_data.username)
    if user is None:
        raise HTTPException(status_code=401, detail="User not found")
    return {"message": "This is secure data!", "user": user}